In [262]:
import pandas as pd

In [263]:
# loading raw data to df
df = pd.read_csv('raw_data.csv')
df.head()

,zip_code,commune,province,type_of_property,subtype_of_property,price,building_condition,facade_number,living_area,equipped_kitchen,bedroom_nr,swimming_pool,furnished,open_fire,terrace,garden,plot_surface
0,2600,Berchem,Antwerpen,0,apartment,149000.0,good,NaN,48,installed,1,0,0,0,9,0,0
1,2100,Deurne,Antwerpen,0,apartment,248000.0,good,NaN,91,installed,3,0,0,0,2,0,0
2,2660,Hoboken,Antwerpen,0,apartment,229000.0,as new,2.0,100,0,3,0,0,0,26,0,0
3,1180,Uccle,Bruxelles,0,apartment,470000.0,good,2.0,179,hyper equipped,3,0,0,0,10,0,0
4,2018,Antwerpen,Antwerpen,0,apartment,480000.0,just renovated,2.0,116,installed,2,0,0,0,7,0,0


In [264]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26147 entries, 0 to 26146
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   zip_code             26147 non-null  int64  
 1   commune              26147 non-null  object 
 2   province             26147 non-null  object 
 3   type_of_property     26147 non-null  int64  
 4   subtype_of_property  26147 non-null  object 
 5   price                26111 non-null  float64
 6   building_condition   19457 non-null  object 
 7   facade_number        16785 non-null  float64
 8   living_area          26147 non-null  int64  
 9   equipped_kitchen     26147 non-null  object 
 10  bedroom_nr           26147 non-null  int64  
 11  swimming_pool        26147 non-null  int64  
 12  furnished            26147 non-null  int64  
 13  open_fire            26147 non-null  int64  
 14  terrace              26147 non-null  int64  
 15  garden               26147 non-null 

In [265]:
# Checking for duplicates
duplicates = df.duplicated()
df[duplicates].shape[0]

# There are 1284 duplicates:

1284

In [266]:
#Remove duplicates
df.drop_duplicates(keep='last', inplace=True) # keep='first (instance) is the default, inplace is to make the change in the same df
df.shape[0]

24863

In [267]:
# Checking that duplicates were indeed removed:
duplicates = df.duplicated(keep='last')
print('Duplicates: ' + str(df[duplicates].shape[0]))

Duplicates: 0


In [268]:
# Remove values with missing price
df.dropna(subset=['price'], inplace=True)

In [269]:
# Assign 'unknown' to NaNs in building_condition:
df['building_condition'].fillna('unknown')

0                  good
1                  good
2                as new
3                  good
4        just renovated
              ...      
26142           unknown
26143           unknown
26144           unknown
26145              good
26146              good
Name: building_condition, Length: 24830, dtype: object

In [270]:
print(len(df['subtype_of_property'].value_counts()))
df['subtype_of_property'].value_counts()

36


subtype_of_property
house                      6210
apartment                  6188
apartment unit             5683
house unit                 1625
villa                       837
duplex                      424
ground floor                387
penthouse                   378
mixed use building          353
ground floor unit           348
apartment block             320
flat studio                 306
penthouse unit              292
flat studio unit            239
exceptional property        159
mansion                     148
duplex unit                 142
service flat                 95
town house                   95
country cottage              92
bungalow                     91
loft                         74
service flat unit            54
triplex                      49
kot unit                     49
kot                          44
farmhouse                    38
loft unit                    37
villa unit                   32
chalet                       12
manor house         

In [271]:
# Create function to remove 'unit' form sub type of property, so 'apartment' and 'apartment unit' become one category and so on
def remove_unit(subtype_of_property):
    if 'unit' in subtype_of_property:
        subtype_of_property = subtype_of_property.replace(' unit', '')
    return subtype_of_property

In [272]:
# Apply the function to the sub type col:
df['subtype_of_property'] = df['subtype_of_property'].apply(remove_unit)

In [273]:
# check result
print(len(df['subtype_of_property'].value_counts()))
df['subtype_of_property'].value_counts()

23


subtype_of_property
apartment               11871
house                    7835
villa                     869
ground floor              735
penthouse                 670
duplex                    566
flat studio               545
mixed use building        355
apartment block           320
exceptional property      159
service flat              149
mansion                   148
loft                      111
town house                 98
kot                        93
country cottage            92
bungalow                   91
triplex                    51
farmhouse                  38
chalet                     12
manor house                11
castle                      9
other property              2
Name: count, dtype: int64

In [274]:
# remove the two rows of 'other property'
# Initial number of rows
print(f'Current number of observations: ' + str(df.shape[0]))

# create condition based on which we will remove the rows:
other_properties = df['subtype_of_property'] == 'other property'
# other_properties # returns a series of booleans for the above condition 

df[other_properties] # returns the rows that meet the other_properties condition
revised_df = df[~other_properties] # returns the rows that do NOT meet the other_properties condition 

print('New number of observations: ' + str(revised_df.shape[0]))

Current number of observations: 24830
New number of observations: 24828


In [275]:
# The sub type of properties is now good to go.
# However, 'equipped_kitchen' still needs to be changed to a numerical value: No : 0, Yes : 1
revised_df['equipped_kitchen'].value_counts()

equipped_kitchen
0                     8770
installed             7397
hyper equipped        5473
semi equipped         1899
not installed          822
usa hyper equipped     273
usa installed          165
usa semi equipped       19
usa uninstalled         10
Name: count, dtype: int64

In [276]:
# create function to make the change:
def correct_kitchen_value(equipped_kitchen):
    if equipped_kitchen in (0, 'not installed', 'usa uninstalled'):
        equipped_kitchen = 0
    elif equipped_kitchen in ('semi equipped','usa semi equipped'):
        equipped_kitchen = 1
    else:
        equipped_kitchen = 2
    return equipped_kitchen


In [277]:
# apply the function to the df col
df['equipped_kitchen'] = df['equipped_kitchen'].apply(correct_kitchen_value)

In [278]:
# check that it is fixed
df['equipped_kitchen'].value_counts()

equipped_kitchen
2    22080
1     1918
0      832
Name: count, dtype: int64

In [ ]:
# Fix NaN under facades by assigning the median value per property sub type
# Find facades per property type:
median_facade_per_p_type = df.groupby(by='subtype_of_property')['facade_number'].median()
median_facade_per_p_type


subtype_of_property
apartment               2.0
apartment block         2.0
bungalow                4.0
castle                  4.0
chalet                  4.0
country cottage         4.0
duplex                  2.0
exceptional property    4.0
farmhouse               4.0
flat studio             2.0
ground floor            2.0
house                   3.0
kot                     2.0
loft                    2.0
manor house             4.0
mansion                 2.0
mixed use building      2.0
other property          4.0
penthouse               2.0
service flat            2.0
town house              2.0
triplex                 2.0
villa                   4.0
Name: facade_number, dtype: float64

In [ ]:
# Assign the median number of facades of each property sub type to the NaN of its group:
df['facade_number'] = df['facade_number'].fillna(df.groupby('subtype_of_property')['facade_number'].transform('median'))

In [299]:
# Check there are no NaN under fa
print(df.shape[0])
df['facade_number'].info()

24830
<class 'pandas.core.series.Series'>
Index: 24830 entries, 0 to 26146
Series name: facade_number
Non-Null Count  Dtype  
--------------  -----  
24830 non-null  float64
dtypes: float64(1)
memory usage: 904.0 KB


In [302]:
# Check houses with 0 bedrooms to find possible causes
df.loc[(df['type_of_property'] == 1) & (df['bedroom_nr'] == 0)].head()
# After some searching, it seems they are mixed-used businesses, projects and even warehouses

,zip_code,commune,province,type_of_property,subtype_of_property,price,building_condition,facade_number,living_area,equipped_kitchen,bedroom_nr,swimming_pool,furnished,open_fire,terrace,garden,plot_surface
15863,1300,Wavre,Brabant Wallon,1,apartment block,180000.0,just renovated,2.0,78,1,0,0,1,0,6,6,100
16079,2140,Borgerhout,Antwerpen,1,apartment block,449900.0,good,2.0,237,2,0,0,0,0,0,0,75
16265,9040,Sint-Amandsberg,Oost-Vlaanderen,1,house,420000.0,good,3.0,234,2,0,0,0,0,8,130,130
16923,9000,Gent,Oost-Vlaanderen,1,mansion,1985000.0,good,2.0,350,2,0,0,0,0,0,0,350
17042,1080,Molenbeek-Saint-Jean,Bruxelles,1,apartment block,599000.0,good,2.0,350,2,0,0,0,0,0,0,0


In [ ]:
# drop rows where living_area is 0